# Bloodrager
Leshy Bloodrager barbarian, critfisher build

**Feats:** **1.** Extended Reach, **2.** Bloodrager Dedication, **4.** Rising Blood Magic, **6.** Siphon Magic, **10.** Hematocritical, **12.** Surging Blood Magic, **18.** Exultant Blood Magic

**Spells**: **cantrips** ignition or live wire, electric arc; **1st** Sure Strike; **2nd** Brine Dragon Bile; **3rd** Haste or Blazing Bolt or Breathe Fire (signature)

**Equipment:** Rooting Flaming Greatpick or Greatsword, (Greater) Phantasmal Doorknob

In [ ]:
import xarray

import pathfinder2e_stats as pf2
from pathfinder2e_stats.tables import PC, SIMPLE_NPC

In [ ]:
level = 14

spell_slot_rank = (
    pf2.level2rank(level, dedication=True) - 1
)  # max - 1, recoverable with Siphon Magic
use_hematocritical = level >= 10
use_rooting_rune = level >= 7  # clumsy 1 on a crit
use_flaming_rune = level >= 10
use_sword = False  # off-guard to ranged spells on a crit
use_greater_phantasmal_doorknob = level >= 10  # off-guard to ranged spells on a crit

## Attak bonus progrssion
Weapon vs. spell

In [ ]:
weapon_atk_bonus = (
    PC.level
    + PC.weapon_proficiency.martial
    + PC.ability_bonus.boosts.sel(initial=4)
    + PC.ability_bonus.apex
    + PC.attack_item_bonus.potency_rune
)

spell_atk_bonus = (
    PC.level
    + PC.spell_proficiency.dedication
    + PC.ability_bonus.boosts.sel(initial=2)
)

atk_bonus = xarray.concat([weapon_atk_bonus, spell_atk_bonus], dim="kind").T
atk_bonus.coords["kind"] = ["weapon", "spell"]
atk_bonus.to_pandas()

### Let's select 3 standard targets:
- level -2 henchman, all defenses are low
- at-level monster, all defenses are moderate
- level +2 boss, all defenses are high

In [ ]:
rank = pf2.level2rank(level)
defenses = SIMPLE_NPC[["AC", "saving_throws"]].sel(level=level)
AC = defenses.AC
saves = defenses.saving_throws
defenses.to_array("kind").to_pandas()

## Build damage profiles for weapon and spells

In [ ]:
STR = (
    + PC.ability_bonus.boosts.sel(initial=4)
    + PC.ability_bonus.apex
).sel(level=level).item()
weapon_specialization = PC.weapon_specialization.martial.sel(level=level).item()
rage_weapon = PC.rage.bloodrager_weapon.sel(level=level).item()
weapon_dmg_bonus = STR + weapon_specialization + rage_weapon
rage_bleed = PC.rage.bloodrager_bleed.sel(level=level).item()
weapon_dice = PC.weapon_dice.striking_rune.sel(level=level).item()

if use_sword:
    # Greatsword with extended reach
    weapon = pf2.armory.swords.greatsword(weapon_dice, weapon_dmg_bonus).reduce_die()
else:
    # Greatpick with extended reach
    weapon = pf2.armory.picks.greatpick(weapon_dice, weapon_dmg_bonus).reduce_die()
    if level >= 5:
        weapon += pf2.armory.picks.critical_specialization(2)

if use_flaming_rune:
    weapon += pf2.armory.runes.flaming()

weapon += pf2.Damage("bleed", 0, 0, rage_bleed, persistent=True)
weapon

In [ ]:
def rage_spell(
    level: int, type_: str, *, persistent: bool = False, drained: int = 2
) -> dict[pf2.DoS, list[pf2.Damage]]:
    raw = PC.rage.bloodrager_spells.sel(level=level, drained=drained).item()
    d = pf2.Damage(type_, 0, 0, raw, persistent=persistent)
    return {
        pf2.DoS.critical_success: [d.copy(multiplier=2)],
        pf2.DoS.success: [d],
        pf2.DoS.failure: [d],
    }


ignition_melee = pf2.armory.cantrips.ignition(rank, melee=True) + rage_spell(
    level, "fire"
)
ignition_melee

In [ ]:
ignition_ranged = pf2.armory.cantrips.ignition(rank, melee=False) + rage_spell(
    level, "fire"
)
ignition_ranged

In [ ]:
live_wire = pf2.armory.cantrips.live_wire(rank) + rage_spell(level, "electricity")
live_wire

In [ ]:
electric_arc = pf2.armory.cantrips.electric_arc(rank)
electric_arc

In [ ]:
breathe_fire = pf2.armory.spells.breathe_fire(spell_slot_rank)
breathe_fire

In [ ]:
brine_dragon_bile = pf2.armory.spells.brine_dragon_bile(spell_slot_rank) + rage_spell(
    level, "acid", persistent=True
)
brine_dragon_bile

In [ ]:
blazing_bolt_1action = pf2.armory.spells.blazing_bolt(
    spell_slot_rank, actions=1
) + rage_spell(level, "fire")
blazing_bolt_1action

In [ ]:
blazing_bolt_3actions = pf2.armory.spells.blazing_bolt(
    spell_slot_rank, actions=3
) + rage_spell(level, "fire")
blazing_bolt_3actions

## Attack routine
- Strike (with flank) -> Hematocritical if crit -> spell, _or_
- (if hasted) Sure Strike -> Strike (with flank) -> Hematocritical -> spell

Spell is one of:
- Ignition (melee with flank)
- Ignition (ranged due to reach)
- Live Wire
- Electric Arc (1-2 targets)
- Breathe Fire / Fireball
- Blazing Bolt (1-2-3 actions)
- (out of round) Brine Dragon Bile

Spells from slots are at maximum rank -1, so that they can be cycled with Syphon Magic.

In [ ]:
sure_strike = xarray.DataArray(
    [False, True, False],
    dims=["Sure Strike"],
    coords={"Sure Strike": ["Normal", "Sure Strike", "Only on melee crit"]},
)

strike = pf2.damage(
    pf2.check(
        bonus=atk_bonus.sel(level=level, kind="weapon").values.tolist(),
        DC=AC - 2,
        fortune=sure_strike,
    ),
    weapon,
)

### What are the chances of a critical hit on the initial weapon strike?

In [ ]:
melee_crit = strike.outcome == pf2.DoS.critical_success
melee_crit.loc[{"Sure Strike": "Only on melee crit"}] = True
melee_crit.mean("roll").round(3).to_pandas() * 100.0

### The conditions of the next spell change depending on the strike and equipment
- if the strike was critical, we can use Hematocritical
- if the weapon was rooting, the target is now Clumsy 1
- if the weapon was a sword, th target is now off-guard even if not flanked

In [ ]:
hematocritical = melee_crit if use_hematocritical else xarray.DataArray(False)
clumsy = melee_crit if use_rooting_rune else xarray.DataArray(0)
ranged_off_guard = (
    2 * melee_crit
    if (use_sword or use_greater_phantasmal_doorknob)
    else xarray.DataArray(0)
)

### Roll damage for the spells

In [ ]:
ignition_melee_dmg = pf2.damage(
    pf2.check(
        spell_atk_bonus.sel(level=level, drop=True) - 5,
        DC=AC - 2 - clumsy,
        fortune=hematocritical,
    ),
    ignition_melee,
)

ignition_ranged_dmg = pf2.damage(
    pf2.check(
        spell_atk_bonus.sel(level=level, drop=True) - 5,
        DC=AC - clumsy - ranged_off_guard,
        fortune=hematocritical,
    ),
    ignition_ranged,
)

live_wire_dmg = pf2.damage(
    pf2.check(
        spell_atk_bonus.sel(level=level, drop=True) - 5,
        DC=AC - clumsy - ranged_off_guard,
        fortune=hematocritical,
    ),
    live_wire,
)

target = xarray.DataArray(
    [1, 0, 0],
    dims=["target"],
    coords={"target": ["Strike target", "target 2", "target 3"]},
)
electric_arc_dmg = pf2.damage(
    pf2.check(
        saves - target[:2] * clumsy,
        DC=spell_atk_bonus.sel(level=level, drop=True) + 10,
        misfortune=hematocritical,
    ),
    electric_arc,
)

breathe_fire_dmg = pf2.damage(
    pf2.check(
        saves - target * clumsy,
        DC=spell_atk_bonus.sel(level=level, drop=True) + 10,
        misfortune=hematocritical,
    ),
    breathe_fire,
)

In [ ]:
blazing_bolt_check = pf2.check(
    spell_atk_bonus.sel(level=level, drop=True) - 5,
    DC=AC - (clumsy + ranged_off_guard) * target,
    fortune=hematocritical,
)
blazing_bolt_1action_dmg = pf2.damage(
    blazing_bolt_check.isel(target=0, drop=True),
    blazing_bolt_1action,
)
blazing_bolt_23actions_dmg = pf2.damage(
    blazing_bolt_check,
    blazing_bolt_3actions,
)

### Also show:
- A second iterative strike
- a standalone 3-actions Blazing Bolt
- an out-of-round Brine Dragon Bile

In [ ]:
strike2 = pf2.damage(
    pf2.check(
        bonus=atk_bonus.sel(level=level, kind="weapon").values.tolist() - 5,
        DC=AC - 2 - clumsy,
    ),
    weapon,
)

blazing_bolt_23actions_noMAP_dmg = pf2.damage(
    pf2.check(
        spell_atk_bonus.sel(level=level, drop=True),
        DC=AC,
        dims={"target": 3},
    ),
    blazing_bolt_3actions,
)

brine_dragon_bile_dmg = pf2.damage(
    pf2.check(
        spell_atk_bonus.sel(level=level, drop=True),
        DC=AC,
    ),
    brine_dragon_bile,
)

## Mean damage for every action

In [ ]:
rows = {
    "Weapon Strike (flanked)": strike,
    "Iterative Weapon Strike (flanked) (MAP-5)": strike2,
    "Ignition (melee, flanked) (MAP-5)": ignition_melee_dmg,
    "Ignition (ranged) (MAP-5)": ignition_ranged_dmg,
    "Live Wire (MAP-5)": live_wire_dmg,
    "Electric Arc (1 target)": electric_arc_dmg.isel(target=slice(1)),
    "Electric Arc (2 targets)": electric_arc_dmg,
    "Breathe Fire (1 target)": breathe_fire_dmg.isel(target=slice(1)),
    "Breathe Fire (2 targets)": breathe_fire_dmg.isel(target=slice(2)),
    "Breathe Fire (3 targets)": breathe_fire_dmg,
    "Blazing Bolt > (MAP-5)": blazing_bolt_1action_dmg,
    "Blazing Bolt >> (MAP-5)": blazing_bolt_23actions_dmg.isel(target=slice(2)),
    "Blazing Bolt >>> (MAP-5)": blazing_bolt_23actions_dmg,
    "Blazing Bolt >>> (standalone)": blazing_bolt_23actions_noMAP_dmg,
    "Brine Dragon Bile (standalone)": brine_dragon_bile_dmg,
}

damages = []
for dmg in rows.values():
    dmg = dmg.total_damage.mean("roll")
    if "target" in dmg.dims:
        dmg = dmg.sum("target")
    damages.append(dmg)

total_damage = xarray.concat(damages, dim="activity", coords="minimal")
total_damage.coords["activity"] = list(rows)

total_damage.loc[
    {"activity": total_damage.activity[0], "Sure Strike": "Only on melee crit"}
] = float("nan")
total_damage.loc[
    {"activity": total_damage.activity[-2:], "Sure Strike": "Only on melee crit"}
] = float("nan")
total_damage.loc[
    {"activity": total_damage.activity[-3:], "Sure Strike": "Sure Strike"}
] = float("nan")
total_damage.stack(col=["challenge", "Sure Strike"]).to_pandas().round(1)

### Outcome probability for the initial Strike

In [ ]:
(
    pf2.outcome_counts(strike)
    .isel({"Sure Strike": slice(2)})
    .stack(col=["challenge", "Sure Strike"])
    .to_pandas()
    .round(3)
    * 100.0
)

### Outcome probability for the iterative Strike

In [ ]:
(
    pf2.outcome_counts(strike2)
    .stack(col=["challenge", "Sure Strike"])
    .to_pandas()
    .round(3)
    * 100.0
)

### Outcome probability for Ignition (melee)

In [ ]:
(
    pf2.outcome_counts(ignition_melee_dmg)
    .stack(col=["challenge", "Sure Strike"])
    .to_pandas()
    .round(3)
    * 100.0
)

### Outcome probability for Electric Arc / Breathe Fire / Fireball

In [ ]:
(
    pf2.outcome_counts(electric_arc_dmg)
    .stack(row=["target", "outcome"])
    .stack(col=["challenge", "Sure Strike"])
    .to_pandas()
    .round(3)
    * 100.0
)